In [1]:
import torch

In [2]:
from torch import nn, optim

In [3]:
from torch.utils import data

In [4]:
from loguru import logger

In [5]:
import visualize

In [6]:
import numpy as np

In [7]:
import time

In [8]:
import torchvision
import torchvision.transforms

In [9]:
def validate(net, val_data, batch_size, num_class):
    net.eval()

    val_data_iter = data.DataLoader(val_data, batch_size=batch_size, shuffle=False)
    use_cuda = net.use_cuda
    
    total_count = torch.zeros(size=(num_class,), dtype=torch.float32)
    true_count = torch.zeros(size=(num_class,), dtype=torch.float32)
    with torch.no_grad():
        for batch_num, (val_data, label) in enumerate(val_data_iter):
            if use_cuda:
                val_data = val_data.cuda()
                label = label.cuda()
            prediction = net(val_data)
            _, pred = torch.max(prediction, 1)
            predict_tensor = (pred == label).squeeze()
            for i, true_label in enumerate(label):
                total_count[true_label] += 1
                true_count[true_label] += predict_tensor[i].item()
            
    precision = true_count.sum().item() / total_count.sum()
    average_precision = true_count / total_count
    mean_ap = average_precision.mean().item()
    res_dct = {'precision': precision, 'map': mean_ap}
    return res_dct, average_precision

In [147]:
def get_activation_layer(activation='relu'):
    """

    :param activation:
    :return:
    """
    if activation == 'relu':
        activation_layer = nn.ReLU()
    elif activation == 'leaky_relu':
        activation_layer = nn.LeakyReLU(0.1)
    elif activation == 'sigmoid':
        activation_layer = nn.Sigmoid()
    elif activation == 'selu':
        activation_layer = nn.SELU()
    elif activation == 'elu':
        activation_layer = nn.ELU()
    elif activation == 'tanh':
        activation_layer = nn.Tanh()
    else:
        activation_layer = None

    return activation_layer


def get_conv_block(dimension,
                   in_channels,
                   out_channels,
                   kernel_size,
                   stride,
                   padding,
                   activation = 'relu',
                   batch_normalize=True,
                   dropout=None):
    """

    :param dimension:
    :param in_channels:
    :param out_channels:
    :param kernel_size:
    :param stride:
    :param padding:
    :param activation:
    :param batch_normalize:
    :param dropout:
    :return:
    """
    if batch_normalize:
        bias = False
    else:
        bias = True

    model = nn.Sequential()
    if dimension == 1:
        conv_layer = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                               bias=bias)
        bn_layer = nn.BatchNorm1d(out_channels)
    elif dimension == 2:
        conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                               bias=bias)
        bn_layer = nn.BatchNorm2d(out_channels)
    elif dimension == 3:
        conv_layer = nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                               bias=bias)
        bn_layer = nn.BatchNorm3d(out_channels)
    else:
        raise ValueError(f'dimension value {dimension} is illegal, should be 1, 2, 3')
    model.add_module('convolution', conv_layer)

    activation_layer = get_activation_layer(activation)

    if activation_layer is not None:
        model.add_module('activation', activation_layer)

    if batch_normalize:
        model.add_module('bn', bn_layer)
    elif dropout is not None:
        model.add_module('dropout', nn.Dropout(dropout))

    return model


def get_conv_block1d(in_channels,
                     out_channels,
                     kernel_size=3,
                     stride=1,
                     padding=0,
                     activation='relu',
                     batch_normalize=True,
                     dropout=None):
    """

    :param in_channels:
    :param out_channels:
    :param kernel_size:
    :param stride:
    :param padding:
    :param activation:
    :param batch_normalize:
    :param dropout:
    :return:
    """
    return get_conv_block(1,
                          in_channels,
                          out_channels,
                          kernel_size,
                          stride,
                          padding,
                          activation,
                          batch_normalize,
                          dropout)


def get_conv_block2d(in_channels,
                     out_channels,
                     kernel_size=3,
                     stride=1,
                     padding=0,
                     activation='relu',
                     batch_normalize=True,
                     dropout=None):
    """

    :param in_channels:
    :param out_channels:
    :param kernel_size:
    :param stride:
    :param padding:
    :param activation:
    :param batch_normalize:
    :param dropout:
    :return:
    """
    return get_conv_block(2,
                          in_channels,
                          out_channels,
                          kernel_size,
                          stride,
                          padding,
                          activation,
                          batch_normalize,
                          dropout)


def get_linear_block(in_dim, out_dim, activation='relu', dropout=None):
    """

    :param in_dim:
    :param out_dim:
    :param activation:
    :param dropout:
    :return:
    """
    model = nn.Sequential()
    linear_layer = nn.Linear(in_dim, out_dim)
    model.add_module('linear', linear_layer)

    activation_layer = get_activation_layer(activation)
    if activation_layer is not None:
        model.add_module('activation', activation_layer)

    if dropout is not None:
        model.add_module('dropout', nn.Dropout(dropout))

    return model


class Net(nn.Module):
    """

    """

    def __init__(self, use_cuda=None):
        """

        """
        super().__init__()
        if use_cuda is None:
            self.use_cuda = torch.cuda.is_available()
        else:
            self.use_cuda = use_cuda

        self.conv_0 = get_conv_block2d(3, 6, 5, activation='relu')
        self.conv_1 = get_conv_block2d(6, 16, 5, activation='relu')
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.fc1 = get_linear_block(16 * 5 * 5, 120, activation='relu', dropout=0.5)
        self.fc2 = get_linear_block(120, 84, activation='relu', dropout=0.5)
        self.fc3 = get_linear_block(84, 10, activation=None)

    def forward(self, x):
        """

        :param x:
        :return:
        """
        x = self.pool(self.conv_0(x))
        x = self.pool(self.conv_1(x))
        x = x.reshape(-1, 16 * 5 * 5)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

    @staticmethod
    def loss(prediction, label):
        """

        :param prediction:
        :param label:
        :return:
        """
        return nn.CrossEntropyLoss()(prediction, label)


In [148]:
root = '/data/CIFAR10'

In [149]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [150]:
train_transform = torchvision.transforms.Compose(
    
    [torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomCrop(size=[32,32], padding=4),
    torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.49140089750289917, 0.4821591377258301, 0.4465310275554657),
                                      (0.24702748656272888, 0.24348321557044983, 0.26158758997917175))
    ])

In [151]:
test_transform = torchvision.transforms.Compose(
    
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.49140089750289917, 0.4821591377258301, 0.4465310275554657),
                                      (0.24702748656272888, 0.24348321557044983, 0.26158758997917175))
    ])

In [152]:
train_dataset = torchvision.datasets.CIFAR10(root, transform=train_transform, download=True)

Files already downloaded and verified


In [153]:
test_dataset = torchvision.datasets.CIFAR10(root, train=False, transform=test_transform, download=True)

Files already downloaded and verified


In [154]:
train_dt, valid_dt = data.random_split(train_dataset, (40000, 10000))

In [155]:
net = Net()

In [156]:
learning_rate = 0.001

In [167]:
def learning_rate_schedules(batch_num, initial_lr, warmup_batchs, total_batchs, decrease_schedules):
    """
    """
    if batch_num < warmup_batchs:
        lr = initial_lr * (batch_num+1)/warmup_batchs
    else:
        num = batch_num - warmup_batchs + 1
        lr = 0.5 * (1 + math.cos(math.pi * (num / total_batchs))) * initial_lr
    return lr

In [157]:
# optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [158]:
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [159]:
batch_size = 4096

In [160]:
epoch = 100

In [161]:
use_cuda = net.use_cuda

In [162]:
if use_cuda:
    net = net.cuda()

In [163]:
use_cuda = net.use_cuda

In [164]:
visualizer = visualize.Visualizer('Net')

In [166]:
for e in range(epoch):
    start_time = time.time()
    net.train()
    train_dt, valid_dt = data.random_split(train_dataset, (40000, 10000))
    train_data_iter = data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    running_loss = 0.0
    for i, (train_data, label) in enumerate(train_data_iter):
        
        
        if use_cuda:
            train_data = train_data.cuda()
            label = label.cuda()
        optimizer.zero_grad()
        prediction = net(train_data)
        loss = net.loss(prediction, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    valid_train_res, valid_ap = validate(net, train_dataset, batch_size, num_class=10)
    valid_test_res, test_ap = validate(net, test_dataset, batch_size, num_class=10)
    loss_dct = {'loss': running_loss}
    val_precision = valid_train_res['map']
    val_ap_dct = {classes[i]: v for i, v in enumerate(valid_ap)}
    test_precision = valid_test_res['map']
    logger.info(f'epoch num: {e}, loss: {running_loss}, val_precision: {val_precision}, test_precision: {test_precision}, learning_rate: {learning_rate}, time: {time.time() - start_time}')
    visualizer.plot(loss_dct)
    visualizer.plot({'val_precision': val_precision})
    visualizer.plot(val_ap_dct)
    visualizer.plot({'test_precision': test_precision})

2019-06-25 16:35:17.836 | INFO     | __main__:<module>:24 - epoch num: 0, loss: 11.808576226234436, val_precision: 0.6262799501419067, test_precision: 0.6460999250411987, learning_rate: 0.001, time: 28.800416946411133
2019-06-25 16:35:46.681 | INFO     | __main__:<module>:24 - epoch num: 1, loss: 11.7473863363266, val_precision: 0.6279200315475464, test_precision: 0.6483999490737915, learning_rate: 0.001, time: 28.792062282562256
2019-06-25 16:36:15.485 | INFO     | __main__:<module>:24 - epoch num: 2, loss: 11.709244847297668, val_precision: 0.6262999773025513, test_precision: 0.6442999839782715, learning_rate: 0.001, time: 28.75298023223877
2019-06-25 16:36:44.176 | INFO     | __main__:<module>:24 - epoch num: 3, loss: 11.785438537597656, val_precision: 0.629040002822876, test_precision: 0.6504999995231628, learning_rate: 0.001, time: 28.639344215393066
2019-06-25 16:37:12.924 | INFO     | __main__:<module>:24 - epoch num: 4, loss: 11.76634693145752, val_precision: 0.6271799802780151

In [149]:
torch.save(net.state_dict(), 'audio_net_002.save')

In [2]:
{'loss': 529.4704940319061, 'val_precision': 0.78054, 'test_precision': 0.6408, 'learning_rate': 0.01, 'time': 19.557708978652954}

{'loss': 529.4704940319061,
 'val_precision': 0.78054,
 'test_precision': 0.6408,
 'learning_rate': 0.01,
 'time': 19.557708978652954}